In [1]:
import json
import pandas as pd
from transformers import (
    PreTrainedTokenizerFast,
    AutoModelForCausalLM,
    AutoTokenizer,
    )

# Analyze how models solve programming tasks

## Python

In [3]:
python_dataset = pd.read_parquet("Datasets/Python/train-00000-of-00002.parquet")

python_dataset

,output,instruction,system
0,Here is an example of a nested loop in Python ...,Create a nested loop to print every combinatio...,"You are a Python code analyst, evaluating scri..."
1,The given problem can be solved by iterating t...,Write a function to find the number of distinc...,"As a Python code composer, you craft elegant a..."
2,Here's an example of code that attempts to sol...,Write code that removes spaces and punctuation...,"As a Python code analyst, you investigate and ..."
3,Here is an implementation of the function in P...,Write a function that checks if a given number...,"As a Python code composer, you craft elegant a..."
4,Here's a possible implementation of the method...,Write a method for a string class which replac...,"As a Python code translator, you convert algor..."
...,...,...,...
279753,Concatenates the input files into a single out...,"Given the following Python 3 function, write t...","As a Python code analyst, you investigate and ..."
279754,Copies any files matching filespec from src_di...,Explain what the following Python 3 code does\...,"As a Python code analyst, you investigate and ..."
279755,Minifies the input javascript files to the out...,"Given the following Python 3 function, write t...","As a python code expert, you are capable of cr..."
279756,Splits a large CSS file into several smaller f...,Explain what the following Python 3 code does\...,"As a Python code translator, you convert algor..."


### Fix "output" column

In [15]:
print(python_dataset["output"][279757].split("```")[0])

def compile_less(input_file, output_file):
    """
    Compile a LESS source file. Minifies the output in release mode.
    """
    from .modules import less

    if not isinstance(input_file, str):
        raise RuntimeError('LESS compiler takes only a single input file.')

    return {
        'dependencies_fn': less.less_dependencies,
        'compiler_fn': less.less_compile,
        'input': input_file,
        'output': output_file,
        'kwargs': {},
    }


### Qwen-coder 2.5

In [ ]:
qwen_coder_model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(qwen_coder_model_name)
qwen_coder_model = AutoModelForCausalLM.from_pretrained(
    qwen_coder_model_name,
    torch_dtype="auto",
    device_map="auto"
)

In [ ]:
# for i in range(len(python_dataset)):
for i in range(1):
    system_prompt = python_dataset["system"][i]
    prompt = python_dataset["instruction"][i]

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(responce)